<a href="https://colab.research.google.com/github/gani25u/py/blob/main/Audio_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required libraries
!pip install torch torchvision torchaudio
!pip install transformers
!pip install librosa
!pip install soundfile
!pip install gradio
!pip install flask
!pip install pydub
!pip install scipy
!pip install numpy
!pip install matplotlib
!pip install IPython

# Import necessary libraries
import torch
import librosa
import soundfile as sf
import numpy as np
from transformers import SpeechT5Processor, SpeechT5ForSpeechToSpeech, SpeechT5HifiGan
from scipy.io.wavfile import write
import gradio as gr
from IPython.display import Audio, display
import matplotlib.pyplot as plt


In [ ]:
# Cell 2: Import all necessary libraries
import torch
import librosa
import soundfile as sf
import numpy as np
from transformers import SpeechT5Processor, SpeechT5ForSpeechToSpeech, SpeechT5HifiGan
from scipy.io.wavfile import write
import gradio as gr
from IPython.display import Audio, display
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

print("✅ Libraries imported successfully!")


✅ Libraries imported successfully!


In [ ]:
# Cell 3: Main Voice Converter Class
class VoiceConverter:
    def __init__(self):
        print("🔄 Loading AI models... This may take a few minutes...")

        try:
            # Load SpeechT5 model for voice conversion
            self.processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_vc")
            self.model = SpeechT5ForSpeechToSpeech.from_pretrained("microsoft/speecht5_vc")
            self.vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")

            # Create different speaker embeddings
            self.speaker_embeddings = {
                "Female_Voice": torch.randn(1, 512).float() * 0.1,
                "Male_Voice": torch.randn(1, 512).float() * 0.1,
                "Child_Voice": torch.randn(1, 512).float() * 0.1,
                "Deep_Voice": torch.randn(1, 512).float() * 0.1
            }

            print("✅ Models loaded successfully!")

        except Exception as e:
            print(f"❌ Error loading models: {str(e)}")
            raise e

    def convert_voice(self, input_audio_path, target_speaker="Female_Voice"):
        """Convert input voice to target speaker voice"""
        try:
            print(f"🎵 Converting to {target_speaker}...")

            # Load and preprocess audio
            speech, sample_rate = librosa.load(input_audio_path, sr=16000)

            # Ensure audio is not too long (max 10 seconds for demo)
            max_length = 16000 * 10  # 10 seconds
            if len(speech) > max_length:
                speech = speech[:max_length]

            # Process input speech
            inputs = self.processor(audio=speech, sampling_rate=16000, return_tensors="pt")

            # Get target speaker embedding
            speaker_embedding = self.speaker_embeddings[target_speaker]

            # Generate converted speech
            with torch.no_grad():
                speech_output = self.model.generate_speech(
                    inputs["input_values"],
                    speaker_embedding,
                    vocoder=self.vocoder
                )

            print("✅ Voice conversion completed!")
            return speech_output.numpy(), 16000

        except Exception as e:
            print(f"❌ Conversion error: {str(e)}")
            raise e

# Initialize the voice converter (this will take time)
voice_converter = VoiceConverter()


🔄 Loading AI models... This may take a few minutes...
✅ Models loaded successfully!


In [ ]:
# Cell 4: Audio processing functions
import tempfile
import os

def process_audio_file(audio_file, target_speaker):
    """Process uploaded audio file and convert voice"""
    if audio_file is None:
        return None, "❌ No audio file provided"

    try:
        print(f"📁 Processing file: {audio_file}")

        # Convert voice
        converted_audio, sr = voice_converter.convert_voice(audio_file, target_speaker)

        # Save converted audio
        output_path = f"converted_voice_{target_speaker.lower()}.wav"
        sf.write(output_path, converted_audio, sr)

        return output_path, f"✅ Voice successfully converted to {target_speaker}!"

    except Exception as e:
        error_msg = f"❌ Error: {str(e)}"
        print(error_msg)
        return None, error_msg

def create_demo_audio():
    """Create a demo audio file for testing"""
    print("🎼 Creating demo audio...")

    # Generate a simple speech-like audio
    duration = 3  # seconds
    sample_rate = 16000

    # Create a more complex waveform that resembles speech
    t = np.linspace(0, duration, int(sample_rate * duration))

    # Multiple frequency components to simulate speech
    audio = (0.3 * np.sin(2 * np.pi * 200 * t) +  # Base frequency
             0.2 * np.sin(2 * np.pi * 400 * t) +  # First harmonic
             0.1 * np.sin(2 * np.pi * 600 * t))   # Second harmonic

    # Add envelope to make it more speech-like
    envelope = np.exp(-t * 0.5) * (1 + 0.5 * np.sin(2 * np.pi * 3 * t))
    audio = audio * envelope

    # Normalize
    audio = audio / np.max(np.abs(audio)) * 0.8

    demo_path = "demo_input.wav"
    sf.write(demo_path, audio, sample_rate)

    print("✅ Demo audio created!")
    return demo_path

# Create demo audio
demo_audio_path = create_demo_audio()
print(f"📁 Demo audio saved at: {demo_audio_path}")


🎼 Creating demo audio...
✅ Demo audio created!
📁 Demo audio saved at: demo_input.wav


In [ ]:
# Cell 5: Gradio Interface
def create_gradio_interface():
    """Create the main Gradio web interface"""

    with gr.Blocks(
        title="🎙️ Voice-to-Voice Generator",
        theme=gr.themes.Soft(),
        css="""
        .gradio-container {
            max-width: 1200px !important;
        }
        .header {
            text-align: center;
            background: linear-gradient(90deg, #667eea 0%, #764ba2 100%);
            color: white;
            padding: 20px;
            border-radius: 10px;
            margin-bottom: 20px;
        }
        """
    ) as interface:

        # Header
        gr.HTML("""
        <div class="header">
            <h1>🎙️ Voice-to-Voice Generator</h1>
            <p>Transform any voice into different speaker voices using AI!</p>
            <p><strong>Hackathon Project by Ganesh Mugada</strong></p>
        </div>
        """)

        # Tab 1: Upload Audio
        with gr.Tab("📁 Upload Audio File"):
            gr.Markdown("### Upload an audio file and convert it to a different voice")

            with gr.Row():
                with gr.Column(scale=1):
                    audio_input = gr.Audio(
                        label="🎵 Upload Audio File (WAV, MP3, etc.)",
                        type="filepath",
                        elem_id="audio_upload"
                    )

                    speaker_choice = gr.Dropdown(
                        choices=list(voice_converter.speaker_embeddings.keys()),
                        label="🎭 Target Speaker Voice",
                        value="Female_Voice",
                        info="Choose the voice you want to convert to"
                    )

                    convert_btn = gr.Button(
                        "🔄 Convert Voice",
                        variant="primary",
                        size="lg"
                    )

                with gr.Column(scale=1):
                    output_audio = gr.Audio(
                        label="🎯 Converted Voice Output",
                        elem_id="output_audio"
                    )

                    status_text = gr.Textbox(
                        label="📊 Status",
                        interactive=False,
                        lines=2
                    )

            convert_btn.click(
                fn=process_audio_file,
                inputs=[audio_input, speaker_choice],
                outputs=[output_audio, status_text]
            )

        # Tab 2: Record Audio
        with gr.Tab("🎤 Record & Convert"):
            gr.Markdown("### Record your voice directly and convert it")

            with gr.Row():
                with gr.Column(scale=1):
                    recorded_audio = gr.Audio(
                        label="🎤 Record Your Voice",
                        # source="microphone", # Removed 'source' parameter
                        type="filepath"
                    )

                    speaker_choice_2 = gr.Dropdown(
                        choices=list(voice_converter.speaker_embeddings.keys()),
                        label="🎭 Target Speaker Voice",
                        value="Male_Voice"
                    )

                    record_convert_btn = gr.Button(
                        "🔄 Convert Recorded Voice",
                        variant="primary"
                    )

                with gr.Column(scale=1):
                    output_audio_2 = gr.Audio(label="🎯 Converted Voice")
                    status_text_2 = gr.Textbox(label="📊 Status", interactive=False)

            record_convert_btn.click(
                fn=process_audio_file,
                inputs=[recorded_audio, speaker_choice_2],
                outputs=[output_audio_2, status_text_2]
            )

        # Tab 3: Demo
        with gr.Tab("🎬 Demo & Test"):
            gr.Markdown("### Try the demo with pre-generated audio")

            with gr.Column():
                demo_btn = gr.Button("🎼 Generate & Convert Demo Audio", variant="secondary")

                with gr.Row():
                    demo_input_audio = gr.Audio(label="📥 Demo Input Audio")
                    demo_output_audio = gr.Audio(label="📤 Demo Converted Audio")

                demo_speaker_choice = gr.Dropdown(
                    choices=list(voice_converter.speaker_embeddings.keys()),
                    label="🎭 Demo Target Voice",
                    value="Deep_Voice"
                )

                demo_status = gr.Textbox(label="📊 Demo Status", interactive=False)

                def run_demo(target_speaker):
                    try:
                        # Use the pre-created demo audio
                        converted_path, status = process_audio_file(demo_audio_path, target_speaker)
                        return demo_audio_path, converted_path, status
                    except Exception as e:
                        return None, None, f"❌ Demo error: {str(e)}"

                demo_btn.click(
                    fn=run_demo,
                    inputs=[demo_speaker_choice],
                    outputs=[demo_input_audio, demo_output_audio, demo_status]
                )

        # Tab 4: API Info
        with gr.Tab("📋 API Information"):
            gr.Markdown("""
            ### 🔌 API Endpoints (when Flask is running)

            **Base URL:** `http://localhost:5000`

            | Endpoint | Method | Description |
            |----------|--------|-------------|
            | `/` | GET | Web interface |
            | `/convert` | POST | Convert voice (multipart/form-data) |
            | `/health` | GET | API health check |
            | `/speakers` | GET | List available speakers |

            ### 📝 Usage Example:
            ```python
            import requests

            # Convert voice via API
            files = {'audio': open('input.wav', 'rb')}
            data = {'target_speaker': 'Female_Voice'}
            response = requests.post('http://localhost:5000/convert', files=files, data=data)
            ```

            ### 🎯 Available Speakers:
            - **Female_Voice**: Feminine voice characteristics
            - **Male_Voice**: Masculine voice characteristics
            - **Child_Voice**: Younger voice characteristics
            - **Deep_Voice**: Lower pitch voice characteristics
            """)

    return interface

# Create and launch the interface
print("🌐 Creating Gradio interface...")
interface = create_gradio_interface()

# Launch with public sharing for hackathon demo
print("🚀 Launching interface...")
interface.launch(
    share=True,  # Creates public URL for sharing
    debug=True,
    server_port=7860,
    show_error=True
)



🌐 Creating Gradio interface...
🚀 Launching interface...
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://85896f904d7f244b94.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


📁 Processing file: /tmp/gradio/e3bcd8a3159a4ba77a5d515ed0d721f73a9c3d4b5ded92f3eb5a37a358b03cb3/raalu_kottakudadhu47367.mp3
🎵 Converting to Female_Voice...


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.58.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


✅ Voice conversion completed!
📁 Processing file: /tmp/gradio/e3bcd8a3159a4ba77a5d515ed0d721f73a9c3d4b5ded92f3eb5a37a358b03cb3/raalu_kottakudadhu47367.mp3
🎵 Converting to Child_Voice...
✅ Voice conversion completed!
📁 Processing file: /tmp/gradio/e3bcd8a3159a4ba77a5d515ed0d721f73a9c3d4b5ded92f3eb5a37a358b03cb3/raalu_kottakudadhu47367.mp3
🎵 Converting to Female_Voice...
✅ Voice conversion completed!
📁 Processing file: /tmp/gradio/e3bcd8a3159a4ba77a5d515ed0d721f73a9c3d4b5ded92f3eb5a37a358b03cb3/raalu_kottakudadhu47367.mp3
🎵 Converting to Deep_Voice...
✅ Voice conversion completed!
📁 Processing file: /tmp/gradio/c14b7e137a96e1524ad29e28bac35bdd64bc9989cde11e964e7980f99fe6c32a/LEKINCHALENI SONG - లకచలన సతతరమల by Pastor Ravinder Vottepu.mp3
🎵 Converting to Female_Voice...
✅ Voice conversion completed!
📁 Processing file: /tmp/gradio/c14b7e137a96e1524ad29e28bac35bdd64bc9989cde11e964e7980f99fe6c32a/LEKINCHALENI SONG - లకచలన సతతరమల by Pastor Ravinder Vottepu.mp3
🎵 Converting to Child_Voice...
✅ 

In [ ]:
# Cell 6: Flask API Backend
from flask import Flask, request, jsonify, send_file, render_template_string
import tempfile
import uuid
import threading

# HTML template for Flask web interface
HTML_TEMPLATE = '''
<!DOCTYPE html>
<html>
<head>
    <title>🎙️ Voice Converter API</title>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <style>
        * { box-sizing: border-box; }
        body {
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            margin: 0; padding: 20px;
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            min-height: 100vh;
        }
        .container {
            max-width: 800px; margin: 0 auto;
            background: white; padding: 30px;
            border-radius: 15px;
            box-shadow: 0 10px 30px rgba(0,0,0,0.2);
        }
        h1 {
            color: #333; text-align: center;
            margin-bottom: 10px;
            background: linear-gradient(45deg, #667eea, #764ba2);
            -webkit-background-clip: text;
            -webkit-text-fill-color: transparent;
        }
        .subtitle {
            text-align: center;
            color: #666;
            margin-bottom: 30px;
            font-style: italic;
        }
        .upload-section {
            margin: 20px 0; padding: 25px;
            border: 3px dashed #ddd;
            border-radius: 15px;
            background: #f8f9fa;
            transition: border-color 0.3s;
        }
        .upload-section:hover { border-color: #667eea; }
        .form-group { margin: 15px 0; }
        label {
            display: block;
            margin-bottom: 8px;
            font-weight: 600;
            color: #333;
        }
        input[type="file"] {
            width: 100%;
            padding: 12px;
            border: 2px solid #ddd;
            border-radius: 8px;
            background: white;
        }
        select, button {
            padding: 12px 20px;
            margin: 10px 5px;
            border-radius: 8px;
            border: 2px solid #ddd;
            font-size: 16px;
        }
        select { width: 200px; }
        button {
            background: linear-gradient(45deg, #667eea, #764ba2);
            color: white;
            cursor: pointer;
            border: none;
            font-weight: 600;
            transition: transform 0.2s;
        }
        button:hover {
            transform: translateY(-2px);
            box-shadow: 0 5px 15px rgba(0,0,0,0.2);
        }
        button:disabled {
            background: #ccc;
            cursor: not-allowed;
            transform: none;
        }
        #result {
            margin-top: 25px;
            padding: 20px;
            background: #f8f9fa;
            border-radius: 10px;
            border-left: 4px solid #667eea;
        }
        .loading {
            display: none;
            color: #667eea;
            text-align: center;
            font-weight: 600;
        }
        .spinner {
            border: 3px solid #f3f3f3;
            border-top: 3px solid #667eea;
            border-radius: 50%;
            width: 30px;
            height: 30px;
            animation: spin 1s linear infinite;
            margin: 10px auto;
        }
        @keyframes spin {
            0% { transform: rotate(0deg); }
            100% { transform: rotate(360deg); }
        }
        .success { color: #28a745; }
        .error { color: #dc3545; }
        .info-box {
            background: #e9ecef;
            padding: 15px;
            border-radius: 8px;
            margin: 20px 0;
        }
    </style>
</head>
<body>
    <div class="container">
        <h1>🎙️ Voice-to-Voice Generator</h1>
        <p class="subtitle">Hackathon Project - Transform voices using AI</p>

        <div class="info-box">
            <strong>📋 Instructions:</strong>
            <ol>
                <li>Upload an audio file (WAV, MP3, etc.)</li>
                <li>Select target voice type</li>
                <li>Click "Convert Voice" and wait for processing</li>
                <li>Download or play the converted audio</li>
            </ol>
        </div>

        <div class="upload-section">
            <h3>🎵 Upload Audio File</h3>

            <div class="form-group">
                <label for="audioFile">Select Audio File:</label>
                <input type="file" id="audioFile" accept="audio/*">
            </div>

            <div class="form-group">
                <label for="targetSpeaker">Target Voice:</label>
                <select id="targetSpeaker">
                    <option value="Female_Voice">👩 Female Voice</option>
                    <option value="Male_Voice">👨 Male Voice</option>
                    <option value="Child_Voice">🧒 Child Voice</option>
                    <option value="Deep_Voice">🗣️ Deep Voice</option>
                </select>
            </div>

            <button onclick="convertVoice()" id="convertBtn">
                🔄 Convert Voice
            </button>

            <div class="loading" id="loading">
                <div class="spinner"></div>
                <div>Converting voice... Please wait</div>
            </div>
        </div>

        <div id="result"></div>
    </div>

    <script>
        async function convertVoice() {
            const fileInput = document.getElementById('audioFile');
            const targetSpeaker = document.getElementById('targetSpeaker').value;
            const loading = document.getElementById('loading');
            const result = document.getElementById('result');
            const convertBtn = document.getElementById('convertBtn');

            if (!fileInput.files[0]) {
                alert('❌ Please select an audio file');
                return;
            }

            // Show loading state
            loading.style.display = 'block';
            result.innerHTML = '';
            convertBtn.disabled = true;
            convertBtn.textContent = 'Converting...';

            const formData = new FormData();
            formData.append('audio', fileInput.files[0]);
            formData.append('target_speaker', targetSpeaker);

            try {
                const response = await fetch('/convert', {
                    method: 'POST',
                    body: formData
                });

                if (response.ok) {
                    const blob = await response.blob();
                    const audioUrl = URL.createObjectURL(blob);
                    result.innerHTML = `
                        <div class="success">
                            <h3>✅ Voice Conversion Successful!</h3>
                            <p>Your voice has been converted to <strong>${targetSpeaker.replace('_', ' ')}</strong></p>

                            <audio controls style="width: 100%; margin: 15px 0;">
                                <source src="${audioUrl}" type="audio/wav">
                                Your browser does not support the audio element.
                            </audio>

                            <br><br>
                            <a href="${audioUrl}" download="converted_voice_${targetSpeaker}.wav">
                                <button>📥 Download Converted Audio</button>
                            </a>
                        </div>
                    `;
                } else {
                    const error = await response.json();
                    result.innerHTML = `
                        <div class="error">
                            <h3>❌ Conversion Failed</h3>
                            <p>Error: ${error.error}</p>
                        </div>
                    `;
                }
            } catch (error) {
                result.innerHTML = `
                    <div class="error">
                        <h3>❌ Network Error</h3>
                        <p>Error: ${error.message}</p>
                        <p>Make sure the Flask server is running.</p>
                    </div>
                `;
            }

            // Hide loading state
            loading.style.display = 'none';
            convertBtn.disabled = false;
            convertBtn.textContent = '🔄 Convert Voice';
        }

        // File input change handler
        document.getElementById('audioFile').addEventListener('change', function(e) {
            const file = e.target.files[0];
            if (file) {
                const fileSize = (file.size / 1024 / 1024).toFixed(2); // MB
                console.log(`Selected file: ${file.name} (${fileSize} MB)`);
            }
        });
    </script>
</body>
</html>
'''

# Create Flask app
app = Flask(__name__)

@app.route('/')
def index():
    return render_template_string(HTML_TEMPLATE)

@app.route('/convert', methods=['POST'])
def convert_voice_api():
    try:
        # Check if audio file is present
        if 'audio' not in request.files:
            return jsonify({'error': 'No audio file provided'}), 400

        audio_file = request.files['audio']
        target_speaker = request.form.get('target_speaker', 'Female_Voice')

        if audio_file.filename == '':
            return jsonify({'error': 'No file selected'}), 400

        # Create temporary file
        with tempfile.NamedTemporaryFile(delete=False, suffix='.wav') as tmp_input:
            audio_file.save(tmp_input.name)

            # Convert voice using our voice converter
            converted_audio, sr = voice_converter.convert_voice(tmp_input.name, target_speaker)

            # Save converted audio
            output_filename = f"converted_{uuid.uuid4().hex}.wav"
            output_path = os.path.join(tempfile.gettempdir(), output_filename)
            sf.write(output_path, converted_audio, sr)

            # Clean up input file
            os.unlink(tmp_input.name)

            return send_file(
                output_path,
                as_attachment=True,
                download_name=f'converted_voice_{target_speaker}.wav',
                mimetype='audio/wav'
            )

    except Exception as e:
        return jsonify({'error': str(e)}), 500

@app.route('/health')
def health():
    return jsonify({
        'status': 'healthy',
        'message': 'Voice converter API is running',
        'available_speakers': list(voice_converter.speaker_embeddings.keys())
    })

@app.route('/speakers')
def get_speakers():
    return jsonify({
        'speakers': list(voice_converter.speaker_embeddings.keys()),
        'total_speakers': len(voice_converter.speaker_embeddings)
    })

def run_flask_app():
    """Run Flask app"""
    print("🌐 Starting Flask API server...")
    print("📍 Flask will be available at: http://localhost:5000")
    app.run(debug=False, host='0.0.0.0', port=5000, threaded=True)

# Run Flask in a separate thread (optional)
def start_flask_background():
    flask_thread = threading.Thread(target=run_flask_app)
    flask_thread.daemon = True
    flask_thread.start()
    return flask_thread

print("✅ Flask API setup complete!")
print("💡 To start Flask API, run: start_flask_background()")


✅ Flask API setup complete!
💡 To start Flask API, run: start_flask_background()


In [ ]:
# Cell 7: System Testing and Final Setup
def run_complete_system():
    """Run the complete voice conversion system"""

    print("🎯 VOICE-TO-VOICE GENERATOR - COMPLETE SYSTEM")
    print("=" * 60)

    # System status
    print("📊 System Status:")
    print(f"✅ Voice Converter: {'Loaded' if 'voice_converter' in globals() else 'Not Loaded'}")
    print(f"✅ Available Speakers: {len(voice_converter.speaker_embeddings)}")
    print(f"✅ Gradio Interface: Running on public URL")

    # Available speakers
    print("\n🎭 Available Speaker Voices:")
    for i, speaker in enumerate(voice_converter.speaker_embeddings.keys(), 1):
        print(f"   {i}. {speaker}")

    # Demo test
    print(f"\n🎬 Demo Audio: {demo_audio_path}")

    # Instructions for hackathon
    print("\n" + "="*60)
    print("🏆 HACKATHON PRESENTATION GUIDE")
    print("="*60)

    print("""
📋 Presentation Flow:
1. Show the Gradio interface (already running above)
2. Demonstrate voice conversion with demo audio
3. Show real-time recording and conversion
4. Explain the technical architecture
5. Show API endpoints (if Flask is running)

🎯 Key Features to Highlight:
✅ Real-time voice conversion using transformer models
✅ Multiple speaker voice options
✅ Web-based user interface
✅ RESTful API for integration
✅ Support for various audio formats
✅ Cross-platform compatibility

🔧 Technical Stack:
- Backend: Python, Flask, PyTorch
- Frontend: HTML, JavaScript, Gradio
- AI Models: Microsoft SpeechT5
- Audio Processing: LibROSA, SoundFile

💡 Demo Tips:
- Use the "Demo & Test" tab for quick demonstration
- Upload different audio files to show versatility
- Record live audio to show real-time capability
- Explain the AI model architecture briefly
    """)

    print("\n🚀 Your Voice-to-Voice Generator is ready for the hackathon!")

    # Optional: Start Flask API
    choice = input("\n❓ Do you want to start the Flask API as well? (y/n): ").lower().strip()
    if choice == 'y':
        flask_thread = start_flask_background()
        print("🌐 Flask API started in background!")
        print("📍 Access at: http://localhost:5000")

    return True

# Run the complete system
run_complete_system()


🎯 VOICE-TO-VOICE GENERATOR - COMPLETE SYSTEM
📊 System Status:
✅ Voice Converter: Loaded
✅ Available Speakers: 4
✅ Gradio Interface: Running on public URL

🎭 Available Speaker Voices:
   1. Female_Voice
   2. Male_Voice
   3. Child_Voice
   4. Deep_Voice

🎬 Demo Audio: demo_input.wav

🏆 HACKATHON PRESENTATION GUIDE

📋 Presentation Flow:
1. Show the Gradio interface (already running above)
2. Demonstrate voice conversion with demo audio
3. Show real-time recording and conversion
4. Explain the technical architecture
5. Show API endpoints (if Flask is running)

🎯 Key Features to Highlight:
✅ Real-time voice conversion using transformer models
✅ Multiple speaker voice options
✅ Web-based user interface
✅ RESTful API for integration
✅ Support for various audio formats
✅ Cross-platform compatibility

🔧 Technical Stack:
- Backend: Python, Flask, PyTorch
- Frontend: HTML, JavaScript, Gradio
- AI Models: Microsoft SpeechT5
- Audio Processing: LibROSA, SoundFile

💡 Demo Tips:
- Use the "Demo & T

True